In [1]:
import davesci as ds

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

<p> </br> <b> In legacy, the pending collection could be decided whether from ACH or debit, but in sub2, a pending collection has no payment_method </b></p>

In [3]:
# input table name from aug 16 to aug 31
tableschema = 'SANDBOX.DEV_ASCHMIDLI.'

tablename_all = []
tablename_enable = []

for i in range(16, 32):
    tablename_all.append('SUBS_COHORT_4_2023_08_' + str(i))
    tablename_enable.append('SUBS_COHORT_4_2023_08_' + str(i) + '_ENABLED')
    

In [4]:
user_startdate = '2023-08-16'
cohort_enddate = '2023-08-21'
metrics_enddate = '2023-08-21'

In [5]:
ENDDATE = 31


In [17]:
metrics_startdate = '2023-08-16'
metrics_enddate = '2023-08-21'

In [16]:
# Until user cohort and until metrics 

sql_str = ""
for i in range(16, ENDDATE + 1):
    if i == 30:
        continue
    if i == 16:
        sql_str = ""
    else:
        sql_str += f"""\n union all \n"""
    temp_enddate = '2023-08-' + str(i)
    sql_str = sql_str + f"""
    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_{i} a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_{i}_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_{i}_enabled b
    """

    

In [10]:
con = ds.snowflake_connect()

In [11]:
user = pd.read_sql_query(sql_str, con)

/Users/yanchunyang/Documents/virenv/bearflow/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
ds.write_snowflake_table(user, "sandbox.dev_yyang.sub_cohort4", con, mode='create')

OSError: Forbidden: https://storage.googleapis.com/upload/storage/v1/b/datasci-staging/o
yanchun.yang@dave.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).

In [17]:
print(sql_str)


    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_16 a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_16_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_16_enabled b
    
 union all 

    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_17 a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_17_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_4_2_2023_08_17_enabled b
    
 union all 

    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    

In [29]:
legacy_str = f"""
with users as
(
    select
        a.user_id,
        a.bill_id,
        a.bill_due_date,
        a.payment_method,
        a.is_collect_succeeded,
        a.billing_cycle
    from  ANALYTIC_DB.DBT_MARTS.FCT_MONTHLY_SUBSCRIPTION_USERS a
    join  sandbox.dev_yyang.sub_cohort4 b 
    on a.user_id = b.id
    where a.billing_cycle = '2023-08' and b.label = 'legacy'


),
attempts as (
    select
        id as attempt_id,
        created,
        CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', created::timestamp_ntz) as created_pst,
        subscription_billing_id,
         parse_json(extra):chargeType as chargeType,
        subscription_payment_id
    from APPLICATION_DB.TRANSACTIONS_DAVE.SUBSCRIPTION_COLLECTION_ATTEMPT
    where CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', created::timestamp_ntz) >= date('{metrics_startdate}')
    and CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', created::timestamp_ntz) < date('{metrics_enddate}')
),
payments as (
    select
        a.attempt_id,
        a.created_pst,
        a.subscription_billing_id,
        a.chargeType,
        b.status,
        b.external_processor,
        b.id as payment_id,
        case
            when b.id is null then
                case
                    when chargeType like 'debit charge%'
                        OR chargeType like '%forced debit only%' then 'DEBIT'
                    WHEN chargeType like 'bank charge%'
                        or chargeType like 'balance is above%' then 'ACH'
                    ELSE 'OTHER'
                END
            WHEN b.id is not null then
                case
                    WHEN external_processor like 'SYNAPSEPAY' THEN 'ACH'
                    WHEN external_processor like 'TABAPAY' THEN 'DEBIT'
                    ELSE 'other'
                end
            else 'others'
        end as processor

    from attempts a
    left join APPLICATION_DB.TRANSACTIONS_DAVE.SUBSCRIPTION_PAYMENT b
    on a.subscription_payment_id = b.id

),
final as (
    select
      a.user_id,
      a.bill_id,
      a.bill_due_date,
      a.payment_method,
      a.is_collect_succeeded,
      a.billing_cycle,
      b.attempt_id,
      b.created_pst,
      b.chargeType,
      b.status,
      b.external_processor,
      b.payment_id,
      b.processor
    from users a
    join payments b
    on a.bill_id = b.subscription_billing_id
)
select * from final

"""

In [30]:
legacy = pd.read_sql_query(legacy_str, con)

/Users/yanchunyang/Documents/virenv/bearflow/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [31]:
legacy.shape

(2371, 13)

In [32]:
sub2_str = f"""
WITH sub_new as (
    select * from DAVE.subscription.subscription sub
),
sc_charge_new as (
    select * from DAVE.SUBSCRIPTION.SUBSCRIPTION_CHARGE
),
attempts_new as (
    select * from DAVE.SUBSCRIPTION.subscription_charge_attempt
),
sc_status_new as (
    SELECT * FROM DAVE.SUBSCRIPTION.subscription_charge_status
),
tier_new AS
(
    SELECT * FROM DAVE.subscription.tier
),
payment_method AS
(
    SELECT * FROM DAVE.SUBSCRIPTION.PAYMENT_METHOD_KIND
),
subscriber_new AS (
    SELECT
        sub.USER_ID,
        sub.ID AS SUBSCRIPTION_ID,
        tier.ID as tier_id
    FROM sub_new sub
    JOIN tier_new tier
    ON sub.tier_id = tier.id
    JOIN sandbox.dev_yyang.sub_cohort4 b
    ON sub.user_id = b.id
    WHERE tier.code = 'one_dollar'
        AND  sub._DELETED = FALSE and b.label='sub2'
),
sub_charge_new AS (
    SELECT
        DATE_TRUNC('MONTH',  CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', sc_charge.term_started)) AS billing_cycle,
        DATE(CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', sc_charge.term_started)) as term_started,
        subscriber.user_id,
        sc_charge.id AS subscription_charge_id,
        sc_status.code as collect_status,
        payment.kind,
        sc_charge.unit_cost
    FROM subscriber_new subscriber
    JOIN sc_charge_new sc_charge ON subscriber.subscription_id = sc_charge.subscription_id
    LEFT JOIN sc_status_new sc_status ON sc_charge.subscription_charge_status_id = sc_status.id
    LEFT JOIN payment_method payment ON sc_charge.payment_method_kind_id = payment.id
),
sub_attempts_new AS (
    SELECT
        a.id as subscription_attempt_id,
        s.subscription_charge_id,
        a.created,
        DATE(CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', a.CREATED)) AS attempt_date,
        DATE_TRUNC('month', DATE(CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', a.CREATED))) as monthnum,
        s.billing_cycle,
        s.user_id,
        s.kind as payment_method
    FROM attempts_new a
    JOIN sub_charge_new s on a.subscription_charge_id = s.subscription_charge_id
    WHERE a.created >= date('{metrics_startdate}') and a.created <= date('{metrics_enddate}')
    order by monthnum, attempt_date, created
),
attempt_group_new as
(
    select
        billing_cycle,
        user_id,
        subscription_charge_id,
        attempt_date,
        monthnum,
        payment_method,
        count(distinct subscription_attempt_id) as attempt_cnt
    from sub_attempts_new
    group by 1, 2, 3, 4, 5, 6
),
last_attempt_new as (
    select
        subscription_charge_id,
        date(max(created)) as last_attempt_date
    from attempts_new
    group by 1
),
success_sub_new as (
    select
        sc.billing_cycle,
        sc.term_started,
        sc.user_id,
        sc.subscription_charge_id,
        sc.collect_status,
        sc.unit_cost / 100 as unit_cost,
        la.last_attempt_date
    from sub_charge_new sc
    join last_attempt_new la
    on sc.subscription_charge_id = la.subscription_charge_id
),
final_new as (
    select
        a.billing_cycle,
        a.user_id,
        a.subscription_charge_id,
        a.attempt_date,
        a.monthnum,
        a.attempt_cnt,
        s.unit_cost,
        iff(s.collect_status in ('collected', 'pending_collection'), 1, 0) as if_succeed,
        s.collect_status,
        a.payment_method
    from attempt_group_new a
    left join success_sub_new s
    on a.subscription_charge_id = s.subscription_charge_id
    and a.attempt_date = s.last_attempt_date
)
select * from final_new
"""

In [33]:
sub2 = pd.read_sql_query(sub2_str, con)

/Users/yanchunyang/Documents/virenv/bearflow/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [34]:
sub2.shape

(1081, 10)

In [35]:
legacy.head(10)

,USER_ID,BILL_ID,BILL_DUE_DATE,PAYMENT_METHOD,IS_COLLECT_SUCCEEDED,BILLING_CYCLE,ATTEMPT_ID,CREATED_PST,CHARGETYPE,STATUS,EXTERNAL_PROCESSOR,PAYMENT_ID,PROCESSOR
0,18008094,271386927,2023-08-17,None,0,2023-08,173441992.0,2023-08-17 16:03:29,"""forced debit only""",CANCELED,TABAPAY,154782155.0,DEBIT
1,9208793,267510266,2023-08-18,None,0,2023-08,NaN,NaT,None,None,None,NaN,None
2,18057325,271413681,2023-08-18,None,0,2023-08,NaN,NaT,None,None,None,NaN,None
3,14438126,269640862,2023-08-18,None,0,2023-08,NaN,NaT,None,None,None,NaN,None
4,16537703,270645187,2023-08-16,None,0,2023-08,173361885.0,2023-08-16 16:06:07,"""debit charge only""",CANCELED,TABAPAY,154707092.0,DEBIT
5,17729454,271234757,2023-08-16,None,0,2023-08,173362033.0,2023-08-16 16:06:27,"""forced debit only""",CANCELED,TABAPAY,154707237.0,DEBIT
6,15656215,270218853,2023-08-16,ACH,1,2023-08,173361769.0,2023-08-16 16:05:32,"""balance is above prefer ACH threshold""",COMPLETED,SYNAPSEPAY,154706980.0,ACH
7,8332652,267137875,2023-08-18,None,0,2023-08,NaN,NaT,None,None,None,NaN,None
8,17675578,271206113,2023-08-16,ACH,1,2023-08,173362289.0,2023-08-16 16:07:11,"""balance is above prefer ACH threshold""",COMPLETED,SYNAPSEPAY,154707488.0,ACH
9,9948070,267790622,2023-08-17,ACH,1,2023-08,173473521.0,2023-08-18 02:17:34,"""bank charge only""",COMPLETED,SYNAPSEPAY,154813246.0,ACH


In [93]:
def get_metrics_legacy(df):
    # attempts
    attempts = df.loc[df['ATTEMPT_ID'].notnull(),:]['ATTEMPT_ID'].nunique()
    # users attempts
    users_attempted = df.loc[df['ATTEMPT_ID'].notnull(),:]['USER_ID'].nunique()
    # sucess
    success = df.loc[df['STATUS'] == 'COMPLETED',:]['USER_ID'].nunique()
    # success by payment method 
    success_payments = df.loc[df['STATUS'] == 'COMPLETED',:].groupby(['PAYMENT_METHOD']).agg({'USER_ID': 'nunique'}).reset_index()
    # payment status
    status = df.groupby(['STATUS']).agg({'USER_ID':'nunique'}).reset_index()
    return attempts, users_attempted, success, success_payments, status

In [94]:
attempts, user_attempted, success, success_payments, status = get_metrics_legacy(legacy)

In [95]:
print(attempts)
print(user_attempted)
print(success)
print(success_payments)
print(status)

1388
1354
1201
  PAYMENT_METHOD  USER_ID
0            ACH     1089
1          Debit      112
      STATUS  USER_ID
0   CANCELED       48
1  COMPLETED     1201
2    PENDING       99
3   RETURNED       17


In [80]:
legacy.groupby(['STATUS']).agg({'PAYMENT_ID': 'count'})

,PAYMENT_ID
STATUS,
CANCELED,70
COMPLETED,1201
PENDING,99
RETURNED,17


In [83]:
attempts = legacy['ATTEMPT_ID'].nunique()

In [84]:
attempts

1388

In [81]:
legacy.loc[legacy['STATUS']== 'PENDING',:]

,USER_ID,BILL_ID,BILL_DUE_DATE,PAYMENT_METHOD,IS_COLLECT_SUCCEEDED,BILLING_CYCLE,ATTEMPT_ID,CREATED_PST,CHARGETYPE,STATUS,EXTERNAL_PROCESSOR,PAYMENT_ID,PROCESSOR
43,17188289,270967134,2023-08-18,ACH,1,2023-08,173517509.0,2023-08-18 16:17:27,"""balance is above prefer ACH threshold""",PENDING,SYNAPSEPAY,154855727.0,ACH
94,16169972,270474277,2023-08-18,ACH,1,2023-08,173534175.0,2023-08-18 21:16:16,"""bank charge only (next day same day)""",PENDING,SYNAPSEPAY,154872282.0,ACH
98,12598067,268773793,2023-08-18,ACH,1,2023-08,173516492.0,2023-08-18 16:15:04,"""balance is above prefer ACH threshold""",PENDING,SYNAPSEPAY,154854714.0,ACH
103,14813959,269817983,2023-08-18,ACH,1,2023-08,173540977.0,2023-08-19 01:10:00,"""bank charge only""",PENDING,SYNAPSEPAY,154878679.0,ACH
109,14127595,269499062,2023-08-18,ACH,1,2023-08,173590585.0,2023-08-20 10:27:40,"""bank charge only""",PENDING,SYNAPSEPAY,154926837.0,ACH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,16718859,270730146,2023-08-17,ACH,1,2023-08,173548494.0,2023-08-19 03:31:16,"""bank charge only""",PENDING,SYNAPSEPAY,154885930.0,ACH
2313,16758934,270749556,2023-08-16,ACH,1,2023-08,173535521.0,2023-08-18 22:48:38,"""bank charge only (next day same day)""",PENDING,SYNAPSEPAY,154873617.0,ACH
2338,8564414,267235291,2023-08-16,ACH,1,2023-08,173583011.0,2023-08-20 04:07:53,"""bank charge only""",PENDING,SYNAPSEPAY,154919520.0,ACH
2351,9224581,267517007,2023-08-17,ACH,1,2023-08,173588917.0,2023-08-20 09:17:59,"""bank charge only""",PENDING,SYNAPSEPAY,154925245.0,ACH


In [46]:
#attempt

attempts = legacy.loc[legacy['ATTEMPT_ID'].notnull(),:]['ATTEMPT_ID'].nunique()

In [49]:
# success 
success = legacy.loc[legacy['IS_COLLECT_SUCCEEDED'] == 1,:]['USER_ID'].nunique()

In [50]:
success

1310

In [52]:
payment = legacy.loc[legacy['IS_COLLECT_SUCCEEDED'] == 1,:].groupby(['PAYMENT_METHOD']).agg({'USER_ID': 'nunique'})

In [53]:
payment

,USER_ID
PAYMENT_METHOD,
ACH,1198
Debit,112


In [54]:
sub2.head(10)

,BILLING_CYCLE,USER_ID,SUBSCRIPTION_CHARGE_ID,ATTEMPT_DATE,MONTHNUM,ATTEMPT_CNT,UNIT_COST,IF_SUCCEED,COLLECT_STATUS,PAYMENT_METHOD
0,2023-08-01,7839636,01H81N1VSKDAM693MX053Z8RJ4,2023-08-17,2023-08-01,2,NaN,0,None,external_debit
1,2023-08-01,3939108,01H7Z51HKBRH9T6B63YNYY15B3,2023-08-16,2023-08-01,1,1.0,1,collected,ach
2,2023-08-01,5164387,01H7Z540CQBHVGY7WRC38W64FV,2023-08-16,2023-08-01,1,1.0,1,collected,ach
3,2023-08-01,8241504,01H7Z4GG719MJQR4X8Z5PGEH5C,2023-08-16,2023-08-01,1,1.0,1,collected,ach
4,2023-08-01,13423510,01H7Z4MV6NW4SWDFJQ5QFZ07JV,2023-08-16,2023-08-01,1,1.0,1,collected,ach
5,2023-08-01,7204504,01H7Z4Q4XV19R2GD6MQ9CG9NP0,2023-08-16,2023-08-01,1,1.0,1,collected,ach
6,2023-08-01,15045667,01H7Z4QW7ESN6DM48978TKDTZR,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
7,2023-08-01,10188474,01H7Z55WG32F83N9QV0H407KCR,2023-08-16,2023-08-01,1,1.0,1,collected,ach
8,2023-08-01,1559253,01H7Z4JPP758SG31FW3WEXV0F5,2023-08-16,2023-08-01,1,1.0,1,collected,ach
9,2023-08-01,17014418,01H7Z5DFCYRRK4YJ8KJQDQWN1A,2023-08-16,2023-08-01,1,1.0,1,collected,ach


In [100]:
def get_metrics_sub2(df):
    #attempts 
    attempts = df['ATTEMPT_CNT'].sum()
    # user_attempted
    user_attempted = df.loc[df['ATTEMPT_DATE'].notnull(),:]['USER_ID'].nunique()
    # success 
    success = df.loc[df['COLLECT_STATUS']== 'collected', :]['USER_ID'].nunique()
    # success by payment method
    success_payment = df.loc[df['COLLECT_STATUS']== 'collected', :].groupby(['PAYMENT_METHOD']).agg({'USER_ID':'nunique'}).reset_index()
    # status
    status = df.groupby('COLLECT_STATUS').agg({'USER_ID': 'nunique'}).reset_index()
    return attempts, user_attempted, success, success_payment, status
    

In [101]:
attempt, user_attempted, success, success_payment, status = get_metrics_sub2(sub2)

In [102]:
print(attempt)
print(user_attempted)
print(success)
print(success_payment)
print(status)

1089
1080
885
   PAYMENT_METHOD  USER_ID
0             ach      684
1  external_debit      201
       COLLECT_STATUS  USER_ID
0           collected      885
1  pending_collection      194


In [97]:
attempts = sub2['ATTEMPT_CNT'].sum()

In [98]:
attempts

1089

In [57]:
sub2.shape

(1081, 10)

In [58]:
success = sub2.agg({'IF_SUCCEED': 'sum'})

In [59]:
success

IF_SUCCEED    1079
dtype: int64

In [62]:
payments = sub2.loc[sub2['IF_SUCCEED']==1,:].groupby(['PAYMENT_METHOD']).agg({'IF_SUCCEED':sum})

In [63]:
payments

,IF_SUCCEED
PAYMENT_METHOD,
ach,684
external_debit,201


In [65]:
success_tmp = sub2.loc[sub2['IF_SUCCEED']==1,:]

In [72]:
success_tmp.groupby(['PAYMENT_METHOD']).agg({'SUBSCRIPTION_CHARGE_ID':'count'})

,SUBSCRIPTION_CHARGE_ID
PAYMENT_METHOD,
ach,684
external_debit,201


In [71]:
success_tmp.loc[success_tmp['USER_ID'].isnull(),:]

,BILLING_CYCLE,USER_ID,SUBSCRIPTION_CHARGE_ID,ATTEMPT_DATE,MONTHNUM,ATTEMPT_CNT,UNIT_COST,IF_SUCCEED,COLLECT_STATUS,PAYMENT_METHOD


In [75]:
exception = sub2.loc[(sub2['IF_SUCCEED']==1) & ~(sub2['PAYMENT_METHOD'].isin(['ach', 'external_debit'])),:]

In [77]:
exception.head(10)

,BILLING_CYCLE,USER_ID,SUBSCRIPTION_CHARGE_ID,ATTEMPT_DATE,MONTHNUM,ATTEMPT_CNT,UNIT_COST,IF_SUCCEED,COLLECT_STATUS,PAYMENT_METHOD
6,2023-08-01,15045667,01H7Z4QW7ESN6DM48978TKDTZR,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
25,2023-08-01,15347497,01H7Z5DT21SWR01X1F2AQCE5NQ,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
44,2023-08-01,11983262,01H7Z5KGZJ4ER9ZBQ640NE5D6W,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
48,2023-08-01,4395157,01H7Z4QGBKMA3V9ZHTR87MQQ5W,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
50,2023-08-01,4879451,01H7Z4E5EFW7MAETBXKMWAM8W5,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
54,2023-08-01,5523958,01H7Z4QXHTSXYS5C32NMC5KAH0,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
68,2023-08-01,6512207,01H7Z522THXSVYP8Q28P3Q08NT,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
74,2023-08-01,16980965,01H7Z57H5JV1WWQNCK5WG2H4AM,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
75,2023-08-01,13746736,01H7Z4T7J5D7M2TP4QDVGFB3J9,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None
76,2023-08-01,6655106,01H7Z4G8T4JWSKFWBSNAS1SQG3,2023-08-16,2023-08-01,1,1.0,1,pending_collection,None


In [ ]:
metrics_startdate = '2023-08-16'
metrics_enddate = '2023-08-21'

In [1]:
sql_str = ""
for i in range(13, 16):
    if i == 30:
        continue
    if i == 16:
        sql_str = ""
    else:
        sql_str += f"""\n union all \n"""
    temp_enddate = '2023-08-' + str(i)
    sql_str = sql_str + f"""
    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_{i} a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_{i}_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_{i}_enabled b
    """

In [3]:
print(sql_str)


 union all 

    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_13 a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_13_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_13_enabled b
    
 union all 

    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_14 a 
    left join SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_14_enabled b 
    on a.id = b.id
    where b.id is null
    union all 
    select 
    id,
    primary_bank_account_id,
    due_date,
    'sub2' as label
    from SANDBOX.DEV_ASCHMIDLI.SUBS_COHORT_5_2023_09_14_enabled b
    
 union all 

    select 
    a.id,
    a.primary_bank_account_id,
    a.due_date,
    'legacy' as label
   